In [1]:
import time
import arcpy
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import csv
import sys
pd.set_option("display.max_columns", 100)

In [3]:
# 4525是CGCS_2000_3_Degree_GK_Zone_37的坐标系编码WKID
# 4490是CGCS_2000的WKID

In [5]:
arcpy.env.overwriteOutput = True # 覆盖已存在文件
arcpy.env.workspace = r'E:\project_data\18changsha\gis'
coord_sys = arcpy.SpatialReference(4490)
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference(4525)

# df_pts_den=pd.read_csv(r'E:\project_data\18changsha\2021city1.csv')
# ls_city=df_pts_den['city'].unique().tolist()
# ls0=os.listdir(r'E:\project_basedata\2023data_research\basedata\basic_dbscan_result_filt')
out_lay = 'point_bj_hd_31'
start = time.time()

input_file='E:\\project_data\\18changsha\\ent\\sz_ent.csv'
arcpy.MakeXYEventLayer_management(table=input_file, 
                                  in_x_field='longitude', 
                                  in_y_field='latitude', 
                                  out_layer=out_lay, spatial_reference=coord_sys)
arcpy.CopyFeatures_management('point_bj_hd_31','sz_ent_point')
print('point done')

point done


In [ ]:
outfc='E:\\project_data\\18changsha\\gis\\poi_spatial_join1.shp'
targ_f='E:\\project_data\\18changsha\\gis\\point_total.shp'
outPolygons='E:\\project_data\\18changsha\\gis\\example_polygon1_Buffer.shp'
arcpy.SpatialJoin_analysis(target_features=targ_f,
                           join_features=outPolygons,
                           join_operation='JOIN_ONE_TO_ONE',
                           out_feature_class=outfc,match_option='WITHIN')
arr = arcpy.da.FeatureClassToNumPyArray(outfc, field_names = ('name','real_city','province','longitude','latitude','name_1'))
df_new_xy = pd.DataFrame(arr)
save_file='E:\\project_data\\18changsha\\poi_spatial_join1.csv'
df_new_xy.to_csv(save_file,index=False,mode='w')

In [ ]:
arcpy.SelectLayerByAttribute_management?
arcpy.MakeFeatureLayer_management?
SelectLayerByLocation
arcpy.CreateFishnet_management(outFeatureClass, originCoordinate,
                               yAxisCoordinate, cellSizeWidth, cellSizeHeight,
                               numRows, numColumns, oppositeCoorner, labels,
                               templateExtent, geometryType)
arcpy.CreateFishnet_management(template=)
PointDensity(in_point_features, population_field, {cell_size}, {neighborhood}, {area_unit_scale_factor})
SpatialJoin(target_features, join_features, out_feature_class, {join_operation}, {join_type}, {field_mapping}, {match_option}, {search_radius}, {distance_field_name})


In [ ]:
# 求四级商圈dbscan核心点与点密度对应的筛选阈值（一簇商圈大于多少个点才能算数）

# 根据dbscan结果表格生成点密度文件，筛选一定阈值的点密度然后把点密度面文件join到点上
arcpy.env.overwriteOutput = True # 覆盖已存在文件
arcpy.env.workspace = r'E:\project_basedata\2023data_research\basedata\basic_geofile'
coord_sys = arcpy.SpatialReference(4490)
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference(4525)

df_pts_den=pd.read_excel(r'E:\project_basedata\2023data_research\basedata\basic_point_density\point_density.xlsx')
ls_city=df_pts_den['city'].unique().tolist()
ls0=os.listdir(r'E:\project_basedata\2023data_research\basedata\basic_dbscan_result')
out_lay = 'point_bj_hd_3'
ls_city=ls_city[3:]
for i in ls_city:
    i_=i+'_'
    for j in ls0:
        if i_ in j and j.endswith('100_1_to_4.csv'):
            val_i=df_pts_den.loc[df_pts_den['city']==i,'level_4'].tolist()[0]
            print(i,j,val_i)
            
            # 首先把csv文件转化成点文件
            input_file='E:\\project_basedata\\2023data_research\\basedata\\basic_dbscan_result\\'+j
            arcpy.MakeXYEventLayer_management(table=input_file, 
                                              in_x_field='dianping_shops_2021.longitude', 
                                              in_y_field='dianping_shops_2021.latitude', 
                                              out_layer=out_lay, spatial_reference=coord_sys)
            arcpy.CopyFeatures_management('point_bj_hd_3','point_sh_1')
            print('point done')
            
            # 然后根据点文件生成点密度文件
            arcpy.env.outputCoordinateSystem = arcpy.SpatialReference(4525)
            in_file='E:\\project_basedata\\2023data_research\\basedata\\basic_geofile\\point_sh_1.shp'
            cellSize=200
            height=1
            width=1
            out_file='E:\\project_basedata\\2023data_research\\basedata\\basic_geofile\\'+i_
            myNbr = arcpy.sa.NbrRectangle(height,width, 'Cell')
            outPdens = arcpy.sa.PointDensity(in_file,'NONE',cellSize,myNbr,'SQUARE_KILOMETERS')
            outPdens.save(out_file)
            print('point_density done')
            
            # 栅格取整
            inRaster = out_file
            out_ra='E:\\project_basedata\\2023data_research\\basedata\\basic_geofile\\ra_'+i_
            outInt = arcpy.sa.Int(inRaster)
            outInt.save(out_ra)
            print('point_density int done')
            
            # 栅格转面
            inRaster=out_ra
            outPolygons='E:\\project_basedata\\2023data_research\\basedata\\basic_geofile\\plg_'+i_+'.shp'
            field='VALUE'
            arcpy.RasterToPolygon_conversion(inRaster,outPolygons,'NO_SIMPLIFY',field,'SINGLE_OUTER_PART')
            print('point_density raster to shapefile done')
            
            # 筛选核心点
            arcpy.MakeFeatureLayer_management(in_file,'point_sh_11')
            arcpy.SelectLayerByAttribute_management('point_sh_11','NEW_SELECTION','core_4 <> -1')
            arcpy.CopyFeatures_management('point_sh_11','point_sh_12')
            print('core point done')
            
            # 将点密度面文件与核心点文件做spatial join
            outfc='E:\\project_basedata\\2023data_research\\basedata\\basic_geofile\\'+i+'_join1.shp'
            targ_f='E:\\project_basedata\\2023data_research\\basedata\\basic_geofile\\point_sh_12.shp'
            arcpy.SpatialJoin_analysis(target_features=targ_f,
                                       join_features=outPolygons,
                                       join_operation='JOIN_ONE_TO_ONE',
                                       out_feature_class=outfc,match_option='WITHIN')
            arr = arcpy.da.FeatureClassToNumPyArray(outfc, field_names = ('dianping_s','GRIDCODE'))
            df_new_xy = pd.DataFrame(arr)
            save_file='E:\\project_basedata\\2023data_research\\basedata\\basic_point_density\\'+i+'_density_grid.csv'
            df_new_xy.to_csv(save_file,index=False,mode='w')
            print('core point spatial join done')

In [ ]:
arcpy.Delete_management(outfc)
            arcpy.Delete_management(targ_f)
            arcpy.Delete_management(in_file)
            arcpy.Delete_management(out_file)
            arcpy.Delete_management(out_ra)
            arcpy.Delete_management(outPolygons)
            arcpy.Delete_management('E:\\project_basedata\\2023data_research\\basedata\\basic_geofile\\point_sh_11.shp')

In [22]:
# 求200m*200m单元0阶，1阶，2阶的共现数据
# 用断尾筛选后的dbscan数据进行计算
arcpy.env.overwriteOutput = True # 覆盖已存在文件
arcpy.env.workspace = r'E:\project_basedata\2023data_research\basedata\basic_geofile'
coord_sys = arcpy.SpatialReference(4490)
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference(4525)

df_pts_den=pd.read_excel(r'E:\project_basedata\2023data_research\basedata\basic_point_density\point_density.xlsx')
ls_city=df_pts_den['city'].unique().tolist()
ls0=os.listdir(r'E:\project_basedata\2023data_research\basedata\basic_dbscan_result_filt')
out_lay = 'point_bj_hd_3'
# ls_city=['chongqing']
for i in ls_city:
    i_=i+'_'
    for j in ls0:
        if i_ in j and j.endswith('100_re_1_to_4.csv'):
            print(i,j)
            
             # 首先把csv文件转化成点文件
            input_file='E:\\project_basedata\\2023data_research\\basedata\\basic_dbscan_result_filt\\'+j
            arcpy.MakeXYEventLayer_management(table=input_file, 
                                              in_x_field='dianping_shops_2021.longitude', 
                                              in_y_field='dianping_shops_2021.latitude', 
                                              out_layer=out_lay, spatial_reference=coord_sys)
            arcpy.CopyFeatures_management('point_bj_hd_3','point_sh_1')
            
            # 筛选核心点
            in_file='E:\\project_basedata\\2023data_research\\basedata\\basic_geofile\\point_sh_1.shp'
            arcpy.MakeFeatureLayer_management(in_file,'point_sh_11')
            arcpy.SelectLayerByAttribute_management('point_sh_11','NEW_SELECTION','core_4 <> -1')
            arcpy.CopyFeatures_management('point_sh_11','point_sh_12')
            
            # 根据核心点的范围形成fishnet
            outFeatureClass='E:\\project_basedata\\2023data_research\\basedata\\basic_geofile\\'+i+'_net.shp'
            template_file='E:\\project_basedata\\2023data_research\\basedata\\basic_geofile\\point_sh_12.shp'
            cellSizeWidth = '200'
            cellSizeHeight = '200'
            numRows =  '0'
            numColumns = '0'
            arcpy.CreateFishnet_management(outFeatureClass,template=template_file,cellSizeWidth,
                                           cellSizeHeight,numRows,numColumns,'NO_LABELS','POLYGON')
            
            # 筛选与点有关的网面文件
            fieldmappings = arcpy.FieldMappings('ID','TARGET_FID')
            outfc='E:\\project_basedata\\2023data_research\\basedata\\basic_geofile\\'+i+'_fishjoin.shp'
            targ_f='E:\\project_basedata\\2023data_research\\basedata\\basic_geofile\\point_sh_12.shp'
            arcpy.SpatialJoin_analysis(target_features=outFeatureClass,
                                       join_features=targ_f,
                                       join_operation='JOIN_ONE_TO_ONE',join_type='KEEP_COMMON',
                                       fieldmappings
                                       out_feature_class=outfc,match_option='COMPLETELY_CONTAINS')
            
            # 将筛选出的点文件打上筛选出的面文件的标签。并输出csv
            outfc_1='E:\\project_basedata\\2023data_research\\basedata\\basic_geofile\\'+i+'_corejoin.shp'
            targ_f='E:\\project_basedata\\2023data_research\\basedata\\basic_geofile\\point_sh_12.shp'
            arcpy.SpatialJoin_analysis(target_features=targ_f,
                                       join_features=outfc,
                                       join_operation='JOIN_ONE_TO_ONE',
                                       out_feature_class=outfc_1,match_option='COMPLETELY_CONTAINS')
            arr = arcpy.da.FeatureClassToNumPyArray(outfc_1, field_names = ('dianping_s','TARGET_F_2'))
            df_new_xy = pd.DataFrame(arr)
            save_file='E:\\project_basedata\\2023data_research\\basedata\\basic_cell\\'+i+'_poi_tag.csv'
            df_new_xy.to_csv(save_file,index=False,mode='w')
            
            # 将筛选后的面文件转化为点
            outFeatureClass='E:\\project_basedata\\2023data_research\\basedata\\basic_geofile\\'+i+'_fishpts.shp'
            arcpy.FeatureVerticesToPoints_management(outfc, outFeatureClass, 'ALL')
            
            # 给每个点加上坐标
            arcpy.AddXY_management(outFeatureClass)
            
            # 输出点文件和打上面标签的csv
            arr = arcpy.da.FeatureClassToNumPyArray(outFeatureClass, field_names = ('FID','TARGET_FID','POINT_X','POINT_Y'))
            df_new_xy = pd.DataFrame(arr)
            save_file='E:\\project_basedata\\2023data_research\\basedata\\basic_cell\\'+i+'_fish_pts.csv'
            df_new_xy.to_csv(save_file,index=False,mode='w')

<Result 'true'>

In [ ]:
# 给各等级商圈的
arcpy.env.overwriteOutput = True # 覆盖已存在文件
arcpy.env.workspace = r'E:\project_basedata\2023data_research\basedata\basic_geofile'
coord_sys = arcpy.SpatialReference(4490)
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference(4525)

df_pts_den=pd.read_excel(r'E:\project_basedata\2023data_research\basedata\basic_point_density\point_density.xlsx')
ls_city=df_pts_den['city'].unique().tolist()
ls0=os.listdir(r'E:\project_basedata\2023data_research\basedata\basic_dbscan_result_filt')
out_lay = 'point_bj_hd_3'
for i in ls_city:
    i_=i+'_'
    for j in ls0:
        if i_ in j and j.endswith('100_re_1_to_4.csv'):
            arcpy.Delete_management('point_sh_11')
            arcpy.Delete_management('point_sh_12')
            input_file='E:\\project_basedata\\2023data_research\\basedata\\basic_dbscan_result\\'+j
            arcpy.MakeXYEventLayer_management(table=input_file, 
                                              in_x_field='dianping_shops_2021.longitude', 
                                              in_y_field='dianping_shops_2021.latitude', 
                                              out_layer=out_lay, spatial_reference=coord_sys)
            arcpy.CopyFeatures_management('point_bj_hd_3','point_sh_1')
            point_sh_1='E:\\project_basedata\\2023data_research\\basedata\\basic_geofile\\'+'point_sh_1.shp'
            arcpy.MakeFeatureLayer_management(point_sh_1,'point_sh_11')
            arcpy.SelectLayerByAttribute_management('point_sh_11','NEW_SELECTION','core_4 <> -1')
            arcpy.CopyFeatures_management('point_sh_11','point_sh_12')

In [2]:
# 把点密度join到poi上
arcpy.env.overwriteOutput = True # 覆盖已存在文件
arcpy.env.workspace = r'E:\project_basedata\2023data_research\basedata\basic_geofile'
coord_sys = arcpy.SpatialReference(4490)
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference(4525)

df_pts_den=pd.read_excel(r'E:\project_basedata\2023data_research\basedata\basic_point_density\point_density.xlsx')
ls_city=df_pts_den['city'].unique().tolist()
ls0=os.listdir(r'E:\project_basedata\2023data_research\basedata\basic_dbscan_result')
out_lay = 'point_bj_hd_3'
# ls_city=['shanghai','beijing']
for i in ls_city:
    i_=i+'_'
    for j in ls0:
        if i_ in j and j.endswith('100_1_to_4.csv'):
            arcpy.Delete_management('point_sh_11')
            arcpy.Delete_management('point_sh_12')
            input_file='E:\\project_basedata\\2023data_research\\basedata\\basic_dbscan_result\\'+j
            arcpy.MakeXYEventLayer_management(table=input_file, 
                                              in_x_field='dianping_shops_2021.longitude', 
                                              in_y_field='dianping_shops_2021.latitude', 
                                              out_layer=out_lay, spatial_reference=coord_sys)
            arcpy.CopyFeatures_management('point_bj_hd_3','point_sh_1')
            point_sh_1='E:\\project_basedata\\2023data_research\\basedata\\basic_geofile\\'+'point_sh_1.shp'
            arcpy.MakeFeatureLayer_management(point_sh_1,'point_sh_11')
            arcpy.SelectLayerByAttribute_management('point_sh_11','NEW_SELECTION','core_4 <> -1')
            arcpy.CopyFeatures_management('point_sh_11','point_sh_12')
            
            outfc='E:\\project_basedata\\2023data_research\\basedata\\basic_geofile\\'+i+'_join1.shp'
            join_f='E:\\project_basedata\\2023data_research\\basedata\\basic_geofile\\'+i+'new.shp'
            targ_f='E:\\project_basedata\\2023data_research\\basedata\\basic_geofile\\point_sh_12.shp'
            arcpy.SpatialJoin_analysis(target_features=targ_f,
                                       join_features=join_f,
                                       join_operation='JOIN_ONE_TO_ONE',
                                       out_feature_class=outfc,match_option='WITHIN')
            arr = arcpy.da.FeatureClassToNumPyArray(outfc, field_names = ('dianping_s','GRIDCODE'))
            df_new_xy = pd.DataFrame(arr)
            save_file='E:\\project_basedata\\2023data_research\\basedata\\basic_point_density\\'+i+'_density_grid.csv'
            df_new_xy.to_csv(save_file,index=False,mode='w')

In [35]:
# 注：因为2.7版本限制，可用内存不多，运行前最好清理内容，重新打开程序运行
# 1）python2.7
df_pts_den=pd.read_excel(r'E:\project_basedata\2023data_research\basedata\basic_point_density\point_density.xlsx')

# 2) python2.7
arcpy.env.overwriteOutput = True # 覆盖已存在文件
arcpy.env.workspace = r'E:\project_basedata\2023data_research\basedata\basic_info_newdata'
# dsc = arcpy.Describe(r'E:\project_basedata\2023data_research\New File Geodatabase.gdb\Export_xian')
coord_sys = arcpy.SpatialReference(4490)
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference(4525)
out_lay = 'point_bj_hd_3'
ls_city=df_pts_den['city'].unique().tolist()

# 获取已经完成坐标转换的文件列表
ls0 = os.listdir(r'E:\project_basedata\2023data_research\basedata\basic_xy')

# 获取缩减后的基础数据文件列表
ls1=os.listdir(r'E:\project_basedata\2023data_research\basedata\basic_info_newdata')

# 选择数据更新方式
type_city_ref=raw_input(u'请输入城市数据更新方式（a/b）[a代表全量，b代表增量]：')
# date_stand=raw_input(u'请输入数据包含的时间[如，20230602]：')
start = time.time()
if type_city_ref=='a':
    for i in ls1:
        filename='E:\\project_basedata\\2023data_research\\basedata\\basic_info_newdata\\'+i
        with open(filename,'r') as f:
            csvfile=csv.reader(f)
            for line in csvfile:
                break
        if 'longitude' in line:
            # print(line)
            arcpy.Delete_management('point_bj_hd_3')
            arcpy.MakeXYEventLayer_management(table=i, in_x_field='longitude', 
                                              in_y_field='latitude', out_layer=out_lay, 
                                              spatial_reference=coord_sys)
            arcpy.CopyFeatures_management('point_bj_hd_3',r'E:\project_basedata\2023data_research\New File Geodatabase.gdb\point_bj_hd_1')
            arcpy.AddXY_management(r'E:\project_basedata\2023data_research\New File Geodatabase.gdb\point_bj_hd_1')
            arr = arcpy.da.FeatureClassToNumPyArray(r'E:\project_basedata\2023data_research\New File Geodatabase.gdb\point_bj_hd_1',
                                                    field_names = ('shop_id','POINT_X','POINT_Y'))
            df_new_xy = pd.DataFrame(arr)
            df_new_xy.rename(columns={'shop_id':'dianping_shops_2021_shop_id'},inplace=True)
            name=i[:-19]
            print(i)
            print(df_new_xy.loc[0,'dianping_shops_2021_shop_id'])
            df_new_xy.to_csv('E:\\project_basedata\\2023data_research\\basedata\\basic_xy\\'+name+'_result.csv',index=False,mode='w')
            
        else:
            # print(line)
            arcpy.Delete_management('point_bj_hd_3')
            arcpy.MakeXYEventLayer_management(table=i, in_x_field='dianping_shops_2021.longitude', 
                                              in_y_field='dianping_shops_2021.latitude', out_layer=out_lay, 
                                              spatial_reference=coord_sys)
            arcpy.CopyFeatures_management('point_bj_hd_3',r'E:\project_basedata\2023data_research\New File Geodatabase.gdb\point_bj_hd_1')
            arcpy.AddXY_management(r'E:\project_basedata\2023data_research\New File Geodatabase.gdb\point_bj_hd_1')
            arr = arcpy.da.FeatureClassToNumPyArray(r'E:\project_basedata\2023data_research\New File Geodatabase.gdb\point_bj_hd_1',
                                                    field_names = ('dianping_shops_2021_shop_id','POINT_X','POINT_Y'))
            df_new_xy = pd.DataFrame(arr)
            name=i[:-19]
            print(i)
            print(df_new_xy.loc[0,'dianping_shops_2021_shop_id'])
            df_new_xy.to_csv('E:\\project_basedata\\2023data_research\\basedata\\basic_xy\\'+name+'_result.csv',index=False,mode='w')
        
elif type_city_ref=='b':
    ls_p=[]
    for i in ls_city:
        for j in ls0:
            if i in j:
                ls_p.append(i)
                continue
    ls_city_p = [x for x in ls_city if x not in ls_p]
    for i in ls_city_p:
        for j in ls1:
            if i in j:
                arcpy.MakeXYEventLayer_management(table=j, in_x_field='dianping_shops_2021.longitude', 
                                                  in_y_field='dianping_shops_2021.latitude', 
                                                  out_layer=out_lay, spatial_reference=coord_sys)
                arcpy.CopyFeatures_management('point_bj_hd_3',
                                              r'E:\project_basedata\2023data_research\New File Geodatabase.gdb\point_bj_hd_1')
                arcpy.AddXY_management(r'E:\project_basedata\2023data_research\New File Geodatabase.gdb\point_bj_hd_1')
                arr = arcpy.da.FeatureClassToNumPyArray(r'E:\project_basedata\2023data_research\New File Geodatabase.gdb\point_bj_hd_1',
                                                        field_names = ('dianping_shops_2021_shop_id','POINT_X','POINT_Y'))
                df_new_xy = pd.DataFrame(arr)
                name=i
                str_i='E:\\project_basedata\\2023data_research\\basedata\\basic_xy\\'+name+'_result.csv'
                df_new_xy.to_csv(str_i,index=False,mode='w')
else:
    print(u'error：输入无效。')
    
end = time.time()
runTime = end - start
print('运行时间：')
print(runTime)
# 3）python3
# 4）python3

请输入城市数据更新方式（a/b）[a代表全量，b代表增量]：a
beijingNewData20230519.csv
11199358.0
changchunNewData20230607.csv
100000488.0
changshaNewData20230607.csv
100001939.0
chengduNewData20230602.csv
11107853.0
chongqingNewData20230607.csv
11102934.0
dalianNewData20230607.csv
11223301.0
eerduosiNewData20230607.csv
11559897.0
ezhouNewData20230607.csv
131554056.0
guangzhouNewData20230602.csv
11192904.0
hangzhouNewData20230602.csv
11180371.0
jinanNewData20230607.csv
11102891.0
kunmingNewData20230602.csv
11557442.0
laibinNewData20230607.csv
14026070.0
langfangNewData20230607.csv
11557210.0
linyiNewData20230607.csv
100008353.0
nanjingNewData20230607.csv
11125115.0
pingxiangNewData20230607.csv
1368587764.0
qingdaoNewData20230607.csv
11169781.0
qinhuangdaoNewData20230607.csv
48603038.0
sanmenxiaNewData20230607.csv
11550017.0
shanghaiNewData20230607.csv
11139034.0
shenzhenNewData20230607.csv
100000028.0
shuozhouNewData20230607.csv
100004103.0
taianNewData20230607.csv
48604611.0
taiyuanNewData20230607.csv
58266523.0

In [89]:
arcpy.env.overwriteOutput = True # 覆盖已存在文件
createFC=
filename='E:\\project_basedata\\2023data_research\\basedata\\basic_info_newdata\\'+'shanghaiNewData20230503.csv'
count=0
with open(filename,'r') as f:
    csvfile=csv.reader(f)
    for line in csvfile:
        if count==0:
            index_id=line.index('dianping_shops_2021.shop_id')
            index_x=line.index('dianping_shops_2021.longitude')
            index_y=line.index('dianping_shops_2021.latitude')
            
        if count!=0:
            shopid_i=int(line[index_id])
            x_i=float(line[index_x])
            y_i=float(line[index_y])
            
            print(shopid_i,x_i,y_i)
            
            
        #print(line)
        count+=1
        if count==10:
            break

(11139034, 121.42794, 31.17336)
(11259003, 121.451939, 31.223316)
(11274580, 121.47411, 31.20257)
(11546983, 121.262988, 31.187112)
(11547254, 121.395551, 31.173659)
(11547314, 121.460846, 31.223491)
(11547626, 121.426519, 31.272446)
(11547965, 121.49235, 31.25766)
(11548285, 121.49523, 31.279225)


In [10]:
arcpy.ListFields(dataset='E:\project_basedata\2023data_research\New File Geodatabase.gdb\point_bj_hd_1')

[<Field object at 0x4c128b0[0x17ecf5d8]>,
 <Field object at 0x4c12c90[0x17ecfe00]>,
 <Field object at 0x4c127b0[0x17ecfd70]>,
 <Field object at 0x4c12ab0[0x17ecfdb8]>,
 <Field object at 0x4c12b90[0x17ecf650]>,
 <Field object at 0x4c127d0[0x17ecffc8]>,
 <Field object at 0x17d541b0[0x17ecf140]>,
 <Field object at 0x16d32d10[0x17ecf608]>,
 <Field object at 0x4c12790[0x17ecf668]>,
 <Field object at 0x4c12730[0x17ecf308]>,
 <Field object at 0x4c129d0[0x17ecf5f0]>,
 <Field object at 0x4c12910[0x17ecfde8]>,
 <Field object at 0x4c12970[0x17ed5080]>,
 <Field object at 0x4c126f0[0x17ed5020]>,
 <Field object at 0x4c12c10[0x17ed5098]>]

In [ ]:
# 参考
# ls0 = os.listdir(r'E:\project_basedata\产品名称-属性对应\第三圈层产品标签——经营\169w可经营产品匹配结果\04match_test')
# df_ls0 = pd.DataFrame()
# for i in ls0:
#     if i.endswith('.csv'):
#         # print(i)
#         df_i = pd.read_csv('E:\\project_basedata\\产品名称-属性对应\\第三圈层产品标签——经营\\169w可经营产品匹配结果\\04match_test\\'+i,dtype = 'str')
#         df_ls0 = pd.concat([df_ls0,df_i],axis=0)
#         print(i,len(df_i),len(df_ls0))

In [16]:
# 把数据量大的csv文件完整保存成点shp
arcpy.env.overwriteOutput = True # 覆盖已存在文件
arcpy.env.workspace = r'E:\project_basedata\2023data_research\basedata\basic_info_newdata'
# dsc = arcpy.Describe(r'E:\project_basedata\2023data_research\New File Geodatabase.gdb\Export_xian')
coord_sys = arcpy.SpatialReference(4490)
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference(4525)
out_lay = 'point_bj_hd_3'
arcpy.MakeXYEventLayer_management(table='kunmingNewData20230602.csv', in_x_field='dianping_shops_2021.longitude', 
                                  in_y_field='dianping_shops_2021.latitude', out_layer=out_lay, spatial_reference=coord_sys)
arcpy.CopyFeatures_management('point_bj_hd_3',r'E:\project_basedata\2023data_research\New File Geodatabase.gdb\km_pts_1')

<Result 'E:\\project_basedata\\2023data_research\\basedata\\km_pts_1.shp'>

In [3]:
# 批量把数据量大的csv文件完整保存成点shp
arcpy.env.overwriteOutput = True # 覆盖已存在文件
arcpy.env.workspace = r'E:\project_basedata\2023data_research\basedata\basic_info_newdata'
coord_sys = arcpy.SpatialReference(4490)
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference(4525)
out_lay = 'point_bj_hd_3'

ls0 = os.listdir(r'E:\project_basedata\2023data_research\basedata\basic_info_newdata')
ls_loc=['shanghaiN','shenzhenN']
for i in ls0:
    for j in ls_loc:
        if j in i:
            filename='E:\\project_basedata\\2023data_research\\basedata\\basic_info_newdata\\'+i
            with open(filename,'r') as f:
                csvfile=csv.reader(f)
                for line in csvfile:
                    break
            index_i=i.find('NewData')
            name_i=i[:index_i]
            if 'longitude' in line:
                arcpy.MakeXYEventLayer_management(table=i, in_x_field='longitude',
                                                  in_y_field='latitude', out_layer=out_lay, spatial_reference=coord_sys)
                arcpy.CopyFeatures_management('point_bj_hd_3','E:\\project_basedata\\2023data_research\\New File Geodatabase.gdb\\'+name_i+'_pts_1')
                break
            else:
                arcpy.MakeXYEventLayer_management(table=i, in_x_field='dianping_shops_2021.longitude',
                                                  in_y_field='dianping_shops_2021.latitude', out_layer=out_lay, spatial_reference=coord_sys)
                arcpy.CopyFeatures_management('point_bj_hd_3','E:\\project_basedata\\2023data_research\\New File Geodatabase.gdb\\'+name_i+'_pts_1')
                break

In [18]:
i

'sh_1_to_4_count.csv'

In [ ]:
arcpy.MakeXYEventLayer_management(table='hangzhouNewData20230602.csv', in_x_field='dianping_shops_2021.longitude',
                                          in_y_field='dianping_shops_2021.latitude', out_layer=out_lay, spatial_reference=coord_sys)
        arcpy.CopyFeatures_management('point_bj_hd_3',r'E:\project_basedata\2023data_research\New File Geodatabase.gdb\km_pts_1')

In [4]:
ln

'11233740,1,\xe9\x87\x91\xe9\x91\xab\xe9\x9e\x8b\xe5\xba\x97,,,2,\xe5\x8c\x97\xe4\xba\xac,\xe5\x8c\x97\xe4\xba\xac,\xe5\x8c\x97\xe4\xba\xac,,10,,\xe9\xa1\xba\xe4\xb9\x89\xe5\x8c\xba,\xe9\xa1\xba\xe5\xaf\x86\xe8\xb7\xaf,,20,\xe8\xb4\xad\xe7\x89\xa9,\xe5\x85\xb6\xe4\xbb\x96\xe6\x9c\x8d\xe9\xa5\xb0\xe9\x9e\x8b\xe5\x8c\x85,0.0,116.727193,40.199355,,,,,,0.0,false,,,,,,,2019-10-12,http://www.dianping.com/shop/11233740\n'

In [2]:
df = pd.read_csv(r'E:\project_basedata\2023data_research\basedata\BeijingNewData20230423.csv',dtype='str')

In [3]:
df

,dianping_shops_2021.shop_id_x,dianping_shops_2021.name,dianping_shops_2021.avg_price,dianping_shops_2021.star,dianping_shops_2021.longitude,dianping_shops_2021.latitude,dianping_shops_2021.score_text,大类,中类,中类Ⅱ,小类
0,11199358,东田头皮管理中心,NaN,0.0,116.415839,39.909354,NaN,美肤美体,美体,NaN,美发
1,11228213,雨润芳轩美容美体,NaN,0.0,116.50868,39.89338,NaN,美肤美体,美容,NaN,美容/SPA
2,11230503,浩然靓点橄榄枝美发工作室,NaN,0.0,116.50868,39.89338,NaN,美肤美体,美体,NaN,美发
3,11231113,琳鑫女人缘女子美容院,NaN,0.0,116.63725,40.06478,NaN,美肤美体,美体,NaN,SPA美体
4,11231232,美依泉,NaN,0.0,116.63725,40.06478,NaN,美肤美体,美容,NaN,美容/SPA
5,11231368,顶尖时尚,NaN,3.0,116.63725,40.06478,NaN,美肤美体,美容,NaN,美容/SPA
6,11232655,美源美发,NaN,0.0,116.544363,40.099407,NaN,美肤美体,美体,NaN,美发
7,11232670,晓辉理发店,NaN,0.0,116.610061,40.124263,NaN,美肤美体,美体,NaN,美发
8,11232685,靓丽理发,NaN,0.0,116.660818,40.215644,NaN,美肤美体,美体,NaN,美发
9,11232694,玉娃美容美发,NaN,0.0,116.661636,40.108451,NaN,美肤美体,美体,NaN,美发


In [3]:
# 'beijing_test_1.shp'
arcpy.env.overwriteOutput = True # 覆盖已存在文件
spatRef = arcpy.SpatialReference(4490)
createFC=arcpy.CreateFeatureclass_management(out_path=r'E:\project_basedata\2023data_research\basedata', out_name='beijing_test_1.shp', geometry_type='POINT', spatial_reference=spatRef)
arcpy.AddField_management(r'E:\project_basedata\2023data_research\basedata\beijing_test_1.shp', field_name='shopID', field_type='TEXT')
arcpy.AddField_management(r'E:\project_basedata\2023data_research\basedata\beijing_test_1.shp', field_name='X_p', field_type='DOUBLE')
arcpy.AddField_management(r'E:\project_basedata\2023data_research\basedata\beijing_test_1.shp', field_name='Y_p', field_type='DOUBLE')
iflds = ['shopID','X_p','Y_p']
iCur = arcpy.da.InsertCursor(r'E:\project_basedata\2023data_research\basedata\beijing_test_1.shp', iflds)
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference(4525)

In [4]:
arcpy.env.overwriteOutput = True # 覆盖已存在文件
spatRef = arcpy.SpatialReference(4490)
createFC=arcpy.CreateFeatureclass_management(out_path=r'E:\project_basedata\2023data_research\basedata', out_name='beijing_test_1.shp', geometry_type='POINT', spatial_reference=spatRef)
arcpy.AddField_management(r'E:\project_basedata\2023data_research\basedata\beijing_test_1.shp', field_name='shopID', field_type='TEXT')
arcpy.AddField_management(r'E:\project_basedata\2023data_research\basedata\beijing_test_1.shp', field_name='X_p', field_type='DOUBLE')
arcpy.AddField_management(r'E:\project_basedata\2023data_research\basedata\beijing_test_1.shp', field_name='Y_p', field_type='DOUBLE')
iflds = ['shopID','SHAPE@XY']
iCur = arcpy.da.InsertCursor(r'E:\project_basedata\2023data_research\basedata\beijing_test_1.shp', iflds)
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference(4525)
count = 1

for ln in open(r'E:\project_basedata\2023data_research\basedata\BeijingNewData20230423.csv', 'r').readlines():
    if count > 1:
        ls_i = ln.split(',')
        id_i = ls_i[0]
        # name_i = ls_i[1]
        x_i = ls_i[4]
        y_i = ls_i[5]
        ivals = [id_i,(float(x_i), float(y_i))]
        print(ivals)
        iCur.insertRow(ivals)
    count+=1
    if count == 1000:
        break

['11199358', (116.415839, 39.909354)]
['11228213', (116.50868, 39.89338)]
['11230503', (116.50868, 39.89338)]
['11231113', (116.63725, 40.06478)]
['11231232', (116.63725, 40.06478)]
['11231368', (116.63725, 40.06478)]
['11232655', (116.544363, 40.099407)]
['11232670', (116.610061, 40.124263)]
['11232685', (116.660818, 40.215644)]
['11232694', (116.661636, 40.108451)]
['11232703', (116.555855, 40.077793)]
['11232704', (116.701538, 40.098438)]
['11232715', (116.700751, 40.095857)]
['11232722', (116.659666, 40.131852)]
['11232728', (116.609689, 40.126128)]
['11232743', (116.661967, 40.135534)]
['11232747', (116.55094, 40.0784)]
['11232755', (116.65595, 40.14009)]
['11232768', (116.652029, 40.174542)]
['11232798', (116.82133, 40.160078)]
['11232804', (116.715506, 40.040458)]
['11232810', (116.65135, 40.13937)]
['11232832', (116.716209, 40.040032)]
['11232855', (116.652054, 40.138042)]
['11232944', (116.652388, 40.122084)]
['11232967', (116.65533, 40.217453)]
['11232984', (116.62158, 40.119

In [9]:
count = 1

for ln in open(r'E:\project_basedata\2023data_research\basedata\dianping_beijing_shops_2021.csv', 'r').readlines():
    if count > 1:
        ls_i = ln.split('\t')
        # print(ls_i)
        id_i = ls_i[0]
        x_i = ls_i[19]
        y_i = ls_i[20]
        ivals = [id_i,float(x_i), float(y_i)]
        pts = arcpy.Point(X=float(x_i),Y=float(y_i))
        pt = arcpy.PointGeometry(pts)
        
        arcpy.AddField_management(in_table=pt, field_name='X_p', field_type='DOUBLE')
        arcpy.AddField_management(in_table=pt, field_name='Y_p', field_type='DOUBLE')
        
        arcpy.CalculateField_management(in_table=pt,field='X_p',expression='!shape.centroid.X@METERS!',expression_type='PYTHON_9.3')
        arcpy.CalculateField_management(in_table=pt,field='Y_p',expression='!shape.centroid.Y@METERS!',expression_type='PYTHON_9.3')
        print(pt)
        print(ivals)
        iCur.insertRow(ivals)
    count+=1
    if count == 1000:
        break

<geoprocessing describe geometry object object at 0x2DE8C120>


ExecuteError: Failed to execute. Parameters are not valid.
ERROR 000728: Field X_p does not exist within table
Failed to execute (CalculateField).


In [ ]:
pts = arcpy.Point(X=lng_lat[32][1],Y=lng_lat[32][2],ID=lng_lat[32][0])
pt = arcpy.PointGeometry(pts)

In [ ]:


arcpy.CalculateField_management(in_table=pt,field='X_p',expression='!shape.centroid.X@METERS!',expression_type='PYTHONN_9.3')
arcpy.CalculateField_management(in_table=pt,field='Y_p',expression='!shape.centroid.Y@METERS!',expression_type='PYTHONN_9.3')

In [24]:
ls = ln.split(',')

['11233740',
 '1',
 '\xe9\x87\x91\xe9\x91\xab\xe9\x9e\x8b\xe5\xba\x97',
 '',
 '',
 '2',
 '\xe5\x8c\x97\xe4\xba\xac',
 '\xe5\x8c\x97\xe4\xba\xac',
 '\xe5\x8c\x97\xe4\xba\xac',
 '',
 '10',
 '',
 '\xe9\xa1\xba\xe4\xb9\x89\xe5\x8c\xba',
 '\xe9\xa1\xba\xe5\xaf\x86\xe8\xb7\xaf',
 '',
 '20',
 '\xe8\xb4\xad\xe7\x89\xa9',
 '\xe5\x85\xb6\xe4\xbb\x96\xe6\x9c\x8d\xe9\xa5\xb0\xe9\x9e\x8b\xe5\x8c\x85',
 '0.0',
 '116.727193',
 '40.199355',
 '',
 '',
 '',
 '',
 '',
 '0.0',
 'false',
 '',
 '',
 '',
 '',
 '',
 '',
 '2019-10-12',
 'http://www.dianping.com/shop/11233740\n']

In [13]:
df_2 = pd.read_csv(r'E:\project_basedata\2023data_research\basedata\Export_Output_2_hd.csv',dtype = {'dianping_s':'int'})
df_3 = df_2.head(500)
point = df_3.loc[:, ['dianping_s', 'dianping_6', 'dianping_7']]

In [8]:
df_3

,FID,OBJECTID,Join_Count,TARGET_FID,JOIN_FID,dianping_s,dianping_1,dianping_2,dianping_3,dianping_4,dianping_5,dianping_6,dianping_7,dianping_8,dianping_9,dianpin_10,PAC,NAME,省代码_,省_1,市代码_,市_1,类型_1,x,y
0,0,29,1,29,5,11198005,龙泉寺,密云区其他,景点,名胜古迹,0.0,116.091330,40.103298,0.00,0.00,0.00,110108,海淀区,110000,北京市,110000,北京市,直辖市,3.793421e+07,4.453444e+06
1,1,60,1,60,5,11228746,石刻艺术馆,海淀区,景点,展馆展览,0.0,116.330358,39.944725,0.00,0.00,0.00,110108,海淀区,110000,北京市,110000,北京市,直辖市,3.795566e+07,4.437022e+06
2,2,817,1,817,5,11233526,中国移动,清河,生活服务,移动营业厅,0.0,116.362142,40.031223,0.00,0.00,0.00,110108,海淀区,110000,北京市,110000,北京市,直辖市,3.795780e+07,4.446797e+06
3,3,3721,1,3721,5,11237007,Semir,西北旺,购物,服装,3.0,116.224251,40.076557,0.00,0.00,0.00,110108,海淀区,110000,北京市,110000,北京市,直辖市,3.794573e+07,4.451132e+06
4,4,4159,1,4159,5,11237525,东方鸿睿生态农业,通州区,景点,其他景点,0.0,116.287544,39.927243,0.00,0.00,0.00,110108,海淀区,110000,北京市,110000,北京市,直辖市,3.795212e+07,4.434861e+06
5,5,4723,1,4723,5,11238198,国窖1573,梨园,美食,茶饮果汁,0.0,116.368397,39.982037,0.00,0.00,0.00,110108,海淀区,110000,北京市,110000,北京市,直辖市,3.795867e+07,4.441364e+06
6,6,4871,1,4871,5,11238391,天主教,梨园,景点,名胜古迹,3.5,116.255880,40.053440,6.90,6.90,6.90,110108,海淀区,110000,北京市,110000,北京市,直辖市,3.794858e+07,4.448723e+06
7,7,10253,1,10253,5,11244810,安然纳米汗蒸馆,西三旗,休闲娱乐,洗浴/汗蒸,0.0,116.330664,40.056943,0.00,0.00,0.00,110108,海淀区,110000,北京市,110000,北京市,直辖市,3.795494e+07,4.449493e+06
8,8,11562,1,11562,5,11246390,桃花坞旅游风景区,昌平区,景点,自然风光,3.0,116.292563,40.010183,6.90,6.90,6.90,110108,海淀区,110000,北京市,110000,北京市,直辖市,3.795200e+07,4.444102e+06
9,9,11627,1,11627,5,11246462,新龙集贸市场,清河,美食,更多食品保健,0.0,116.329001,40.056223,0.00,0.00,0.00,110108,海淀区,110000,北京市,110000,北京市,直辖市,3.795480e+07,4.449404e+06


In [9]:
df_filt=pd.read_csv(r'E:\project_basedata\2023data_research\basedata\sh_164_filt.csv')

MemoryError: 

In [10]:
df_auto=pd.read_csv(r'E:\project_basedata\2023data_research\basedata\sh_164_auto.csv',dtype={'dianping_shops_2021.name':'str','dianping_shops_2021.avg_price':'str','dianping_shops_2021.score_text':'str','中类Ⅱ':'str'})
df_auto

C:\Python27\ArcGIS10.8\lib\site-packages\IPython\core\interactiveshell.py:2714: DtypeWarning: Columns (3,7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


MemoryError: 

In [4]:
df=pd.read_csv(r'E:\project_basedata\2023data_research\basedata\newdata20230503\newdata20230503\ShanghaiNewData20230503.csv',
               dtype={'dianping_shops_2021.name':'str','dianping_shops_2021.avg_price':'str','dianping_shops_2021.score_text':'str',
                     '中类Ⅱ':'str'})
df_new=df.loc[:,['dianping_shops_2021.shop_id','dianping_shops_2021.longitude','dianping_shops_2021.latitude']]
df_new.to_csv(r'E:\project_basedata\2023data_research\basedata\newdata20230503\newdata20230503\ShanghaiNewData20230511_filt.csv',index=False)

In [4]:
arcpy.ListFields(r'E:\project_basedata\2023data_research\basedata\dianping_beijing_shops_2021.csv')


[<Field object at 0x1d7d68f0[0x1d7cdc20]>,
 <Field object at 0x1d7b2df0[0x1d7cd3f8]>,
 <Field object at 0x1d7d6590[0x1d7cd218]>,
 <Field object at 0x1d7d67b0[0x1d7cd9c8]>,
 <Field object at 0x4bee6f0[0x1d7cd518]>,
 <Field object at 0x1d390630[0x1d7cd188]>,
 <Field object at 0x1d7b2e70[0x1d7cd6b0]>,
 <Field object at 0x1d7d69d0[0x1d7cd590]>,
 <Field object at 0x1d7d6a10[0x1d7cd140]>,
 <Field object at 0x1d7d6a50[0x1d7cd308]>,
 <Field object at 0x1d7b2cf0[0x1d7cd620]>,
 <Field object at 0x1d6e67f0[0x1d7cfc98]>,
 <Field object at 0x1d2f7150[0x1d7cfcb0]>,
 <Field object at 0x1d7d6af0[0x1d7cfcc8]>,
 <Field object at 0x1d7d6b30[0x1d7cfce0]>,
 <Field object at 0x1d7d6b70[0x1d7cfc80]>,
 <Field object at 0x1d6e68d0[0x1d7cfd10]>,
 <Field object at 0x1d7d6bd0[0x1d7cfd28]>,
 <Field object at 0x1d390790[0x1d7cfcf8]>,
 <Field object at 0x1d7d6c30[0x1d7cfd58]>,
 <Field object at 0x1d7d6c70[0x1d7cfd70]>,
 <Field object at 0x1d390b70[0x1d7cfd88]>,
 <Field object at 0x1d8f8fd0[0x1d7cfd40]>,
 <Field obje

In [ ]:
df_bj = pd.read_csv(r'E:\project_basedata\2023data_research\basedata\dianping_beijing_shops_2021.csv',sep = '\t',
                    dtype = {'dianping_shops_2021.bookable':'str','dianping_shops_2021.has_takeaway':'str','dianping_shops_2021.has_deals':'str'})

In [16]:
df_2.loc[df_2['dianping_s'] == 99044917]

,FID,OBJECTID,Join_Count,TARGET_FID,JOIN_FID,dianping_s,dianping_1,dianping_2,dianping_3,dianping_4,dianping_5,dianping_6,dianping_7,dianping_8,dianping_9,dianpin_10,PAC,NAME,省代码_,省_1,市代码_,市_1,类型_1,x,y
234254,234254,1589566,1,1589566,5,99044917,家乡手擀面,良乡,美食,粉面馆,3.0,116.140358,39.726384,3.57,3.58,3.58,110108,海淀区,110000,北京市,110000,北京市,直辖市,3.795711e+07,4.440549e+06
234255,234255,1589595,1,1589595,5,99044917,家乡手擀面,良乡,美食,粉面馆,3.0,116.140358,39.726384,3.57,3.58,3.58,110108,海淀区,110000,北京市,110000,北京市,直辖市,3.795406e+07,4.440861e+06
234256,234256,1589606,1,1589606,5,99044917,家乡手擀面,良乡,美食,粉面馆,3.0,116.140358,39.726384,3.57,3.58,3.58,110108,海淀区,110000,北京市,110000,北京市,直辖市,3.795210e+07,4.438016e+06
234257,234257,1589619,1,1589619,5,99044917,家乡手擀面,良乡,美食,粉面馆,3.0,116.140358,39.726384,3.57,3.58,3.58,110108,海淀区,110000,北京市,110000,北京市,直辖市,3.794718e+07,4.437536e+06
234258,234258,1589650,1,1589650,5,99044917,家乡手擀面,良乡,美食,粉面馆,3.0,116.140358,39.726384,3.57,3.58,3.58,110108,海淀区,110000,北京市,110000,北京市,直辖市,3.795208e+07,4.437686e+06
234259,234259,1589660,1,1589660,5,99044917,家乡手擀面,良乡,美食,粉面馆,3.0,116.140358,39.726384,3.57,3.58,3.58,110108,海淀区,110000,北京市,110000,北京市,直辖市,3.795725e+07,4.444242e+06
234260,234260,1589661,1,1589661,5,99044917,家乡手擀面,良乡,美食,粉面馆,3.0,116.140358,39.726384,3.57,3.58,3.58,110108,海淀区,110000,北京市,110000,北京市,直辖市,3.795866e+07,4.442070e+06
234261,234261,1589666,1,1589666,5,99044917,家乡手擀面,良乡,美食,粉面馆,3.0,116.140358,39.726384,3.57,3.58,3.58,110108,海淀区,110000,北京市,110000,北京市,直辖市,3.795678e+07,4.438946e+06
234262,234262,1589674,1,1589674,5,99044917,家乡手擀面,良乡,美食,粉面馆,3.0,116.140358,39.726384,3.57,3.58,3.58,110108,海淀区,110000,北京市,110000,北京市,直辖市,3.795622e+07,4.439389e+06
234263,234263,1589680,1,1589680,5,99044917,家乡手擀面,良乡,美食,粉面馆,3.0,116.140358,39.726384,3.57,3.58,3.58,110108,海淀区,110000,北京市,110000,北京市,直辖市,3.795797e+07,4.438045e+06


In [14]:
df_2

,FID,OBJECTID,Join_Count,TARGET_FID,JOIN_FID,dianping_s,dianping_1,dianping_2,dianping_3,dianping_4,dianping_5,dianping_6,dianping_7,dianping_8,dianping_9,dianpin_10,PAC,NAME,省代码_,省_1,市代码_,市_1,类型_1,x,y
0,0,29,1,29,5,11198005,龙泉寺,密云区其他,景点,名胜古迹,0.0,116.091330,40.103298,0.00,0.00,0.00,110108,海淀区,110000,北京市,110000,北京市,直辖市,3.793421e+07,4.453444e+06
1,1,60,1,60,5,11228746,石刻艺术馆,海淀区,景点,展馆展览,0.0,116.330358,39.944725,0.00,0.00,0.00,110108,海淀区,110000,北京市,110000,北京市,直辖市,3.795566e+07,4.437022e+06
2,2,817,1,817,5,11233526,中国移动,清河,生活服务,移动营业厅,0.0,116.362142,40.031223,0.00,0.00,0.00,110108,海淀区,110000,北京市,110000,北京市,直辖市,3.795780e+07,4.446797e+06
3,3,3721,1,3721,5,11237007,Semir,西北旺,购物,服装,3.0,116.224251,40.076557,0.00,0.00,0.00,110108,海淀区,110000,北京市,110000,北京市,直辖市,3.794573e+07,4.451132e+06
4,4,4159,1,4159,5,11237525,东方鸿睿生态农业,通州区,景点,其他景点,0.0,116.287544,39.927243,0.00,0.00,0.00,110108,海淀区,110000,北京市,110000,北京市,直辖市,3.795212e+07,4.434861e+06
5,5,4723,1,4723,5,11238198,国窖1573,梨园,美食,茶饮果汁,0.0,116.368397,39.982037,0.00,0.00,0.00,110108,海淀区,110000,北京市,110000,北京市,直辖市,3.795867e+07,4.441364e+06
6,6,4871,1,4871,5,11238391,天主教,梨园,景点,名胜古迹,3.5,116.255880,40.053440,6.90,6.90,6.90,110108,海淀区,110000,北京市,110000,北京市,直辖市,3.794858e+07,4.448723e+06
7,7,10253,1,10253,5,11244810,安然纳米汗蒸馆,西三旗,休闲娱乐,洗浴/汗蒸,0.0,116.330664,40.056943,0.00,0.00,0.00,110108,海淀区,110000,北京市,110000,北京市,直辖市,3.795494e+07,4.449493e+06
8,8,11562,1,11562,5,11246390,桃花坞旅游风景区,昌平区,景点,自然风光,3.0,116.292563,40.010183,6.90,6.90,6.90,110108,海淀区,110000,北京市,110000,北京市,直辖市,3.795200e+07,4.444102e+06
9,9,11627,1,11627,5,11246462,新龙集贸市场,清河,美食,更多食品保健,0.0,116.329001,40.056223,0.00,0.00,0.00,110108,海淀区,110000,北京市,110000,北京市,直辖市,3.795480e+07,4.449404e+06


In [ ]:
df_bj = pd.read_csv(r'E:\project_basedata\2023data_research\basedata\dianping_beijing_shops_2021.csv',sep = '\t',
                    dtype = {'dianping_shops_2021.bookable':'str','dianping_shops_2021.has_takeaway':'str','dianping_shops_2021.has_deals':'str'})


In [21]:
pd.set_option("display.max_columns", 100)
df_new_m = pd.read_csv(r'E:\project_basedata\2023data_research\basedata\Export_Output_2_hd_0411_m.csv',dtype = 'str')
df_new_lon = pd.read_csv(r'E:\project_basedata\2023data_research\basedata\Export_Output_2_hd_0411_lon.csv',dtype = 'str')

In [31]:
df_new_m.loc[(df_new_m['labels']=='-1')&(df_new_m['core']=='core')]

,FID,OBJECTID,Join_Count,TARGET_FID,JOIN_FID,dianping_s,dianping_1,dianping_2,dianping_3,dianping_4,dianping_5,dianping_6,dianping_7,dianping_8,dianping_9,dianpin_10,PAC,NAME,省代码_,省_1,市代码_,市_1,类型_1,x,y,labels,core,core_label
234257,234257,1589619,1,1589619,5,99044917,家乡手擀面,良乡,美食,粉面馆,3.0,116.140358,39.726384,3.57,3.58,3.58,110108,海淀区,110000,北京市,110000,北京市,直辖市,37947181.361,4437536.36507,-1,core,NaN
234260,234260,1589661,1,1589661,5,99044917,家乡手擀面,良乡,美食,粉面馆,3.0,116.140358,39.726384,3.57,3.58,3.58,110108,海淀区,110000,北京市,110000,北京市,直辖市,37958662.1893,4442069.56135,-1,core,NaN
234266,234266,1589722,1,1589722,5,99044917,家乡手擀面,良乡,美食,粉面馆,3.0,116.140358,39.726384,3.57,3.58,3.58,110108,海淀区,110000,北京市,110000,北京市,直辖市,37954323.7786,4435513.00737,-1,core,NaN
234267,234267,1589731,1,1589731,5,99044917,家乡手擀面,良乡,美食,粉面馆,3.0,116.140358,39.726384,3.57,3.58,3.58,110108,海淀区,110000,北京市,110000,北京市,直辖市,37951365.0348,4433251.42774,-1,core,NaN
234268,234268,1589740,1,1589740,5,99044917,家乡手擀面,良乡,美食,粉面馆,3.0,116.140358,39.726384,3.57,3.58,3.58,110108,海淀区,110000,北京市,110000,北京市,直辖市,37954382.1611,4445643.53191,-1,core,NaN
234270,234270,1589746,1,1589746,5,99044917,家乡手擀面,良乡,美食,粉面馆,3.0,116.140358,39.726384,3.57,3.58,3.58,110108,海淀区,110000,北京市,110000,北京市,直辖市,37949096.0689,4439915.51175,-1,core,NaN
234271,234271,1589749,1,1589749,5,99044917,家乡手擀面,良乡,美食,粉面馆,3.0,116.140358,39.726384,3.57,3.58,3.58,110108,海淀区,110000,北京市,110000,北京市,直辖市,37953347.2696,4434620.77736,-1,core,NaN
234272,234272,1589750,1,1589750,5,99044917,家乡手擀面,良乡,美食,粉面馆,3.0,116.140358,39.726384,3.57,3.58,3.58,110108,海淀区,110000,北京市,110000,北京市,直辖市,37957380.6687,4442999.31772,-1,core,NaN
234273,234273,1589752,1,1589752,5,99044917,家乡手擀面,良乡,美食,粉面馆,3.0,116.140358,39.726384,3.57,3.58,3.58,110108,海淀区,110000,北京市,110000,北京市,直辖市,37954592.3157,4434808.66286,-1,core,NaN
234274,234274,1589755,1,1589755,5,99044917,家乡手擀面,良乡,美食,粉面馆,3.0,116.140358,39.726384,3.57,3.58,3.58,110108,海淀区,110000,北京市,110000,北京市,直辖市,37944470.2245,4456210.40743,-1,core,NaN


In [27]:
ls_3 = df_new_lon.loc[df_new_m['core'].notnull()].index.tolist()
ls_4 = df_new_m.loc[df_new_m['core'].notnull()].index.tolist()

In [28]:
[x for x in ls_4 if x not in ls_3]

[]

In [8]:
df_long = df_new.loc[df_new.duplicated(subset = ['dianping_6','dianping_7'])]

In [9]:
df_m = df_new.loc[df_new.duplicated(subset = ['x','y'])]

In [12]:
ls_1 = df_long.index.tolist()
ls_2 = df_m.index.tolist()


11869

In [ ]:
len([x for x in ls_1 if x not in ls_2])

In [18]:
df_m.iloc[234]

FID                   10418
OBJECTID              86771
Join_Count                1
TARGET_FID            86771
JOIN_FID                  5
dianping_s        123596370
dianping_1               雅衣
dianping_2              四季青
dianping_3               购物
dianping_4             服饰鞋包
dianping_5              0.0
dianping_6       116.270931
dianping_7         39.94374
dianping_8              0.0
dianping_9              0.0
dianpin_10              0.0
PAC                  110108
NAME                    海淀区
省代码_                 110000
省_1                     北京市
市代码_                 110000
市_1                     北京市
类型_1                    直辖市
x             37950586.6744
y             4436609.60123
labels                   -1
core                    NaN
core_label              NaN
Name: 10418, dtype: object

In [20]:
len(df_new.iloc[ls_2_1])

39

In [19]:
ls_2_1 = [x for x in ls_2 if x not in ls_1]
df_new.iloc[ls_2_1]

,FID,OBJECTID,Join_Count,TARGET_FID,JOIN_FID,dianping_s,dianping_1,dianping_2,dianping_3,dianping_4,dianping_5,dianping_6,dianping_7,dianping_8,dianping_9,dianpin_10,PAC,NAME,省代码_,省_1,市代码_,市_1,类型_1,x,y,labels,core,core_label
235242,235242,1597167,1,1597167,5,99002147,原心艺术儿童美术培训中心,梨园,亲子,儿童美术,0.0,116.6823,39.88106,0.0,0.0,0.0,110108,海淀区,110000,北京市,110000,北京市,直辖市,37951434.5191,4433300.37232,-1,NaN,NaN
235262,235262,1597289,1,1597289,5,99007098,汽车洗车美容,管庄,爱车,美容洗车,0.0,116.592354,39.911487,0.0,0.0,0.0,110108,海淀区,110000,北京市,110000,北京市,直辖市,37958412.8105,4442103.68641,-1,NaN,NaN
235265,235265,1597305,1,1597305,5,99007837,摩西尔装饰,朝阳区,家装,装修设计,0.0,116.462085,40.057217,0.0,0.0,0.0,110108,海淀区,110000,北京市,110000,北京市,直辖市,37952058.9958,4438320.42047,1,NaN,NaN
235272,235272,1597367,1,1597367,5,99009630,北京松微文化传媒有限公司,亚运村,生活服务,公司企业,0.0,116.409066,39.999092,0.0,0.0,0.0,110108,海淀区,110000,北京市,110000,北京市,直辖市,37953593.1969,4432835.74218,4,NaN,NaN
235274,235274,1597369,1,1597369,5,99009878,小百花艺术培训中心,大红门,教育培训,国学,0.0,116.38248,39.838754,7.1,7.1,7.1,110108,海淀区,110000,北京市,110000,北京市,直辖市,37958611.731,4440833.61061,3,NaN,NaN
235279,235279,1597388,1,1597388,5,99010875,朝凯五金建材,百子湾,家装,家用五金,0.0,116.491646,39.898646,0.0,0.0,0.0,110108,海淀区,110000,北京市,110000,北京市,直辖市,37957143.3864,4444637.44936,5,NaN,NaN
235280,235280,1597389,1,1597389,5,99010883,施耐德ABB,百子湾,家装,建材,0.0,116.491741,39.89869,0.0,0.0,0.0,110108,海淀区,110000,北京市,110000,北京市,直辖市,37953598.3867,4432806.27777,4,NaN,NaN
235289,235289,1597443,1,1597443,5,99012710,紫荆花油漆专卖店,百子湾,家装,油漆涂料,0.0,116.49166,39.898636,0.0,0.0,0.0,110108,海淀区,110000,北京市,110000,北京市,直辖市,37950586.6744,4436609.60123,-1,NaN,NaN
235294,235294,1597487,1,1597487,5,99030990,Blest Mother蓓塑产后修复,双井,亲子,产后恢复,0.0,116.459261,39.898577,0.0,0.0,0.0,110108,海淀区,110000,北京市,110000,北京市,直辖市,37955707.8241,4441475.29936,2,NaN,NaN
235296,235296,1597494,1,1597494,5,99035351,快修无忧管道疏通,北京南站/开阳里,生活服务,管道疏通,3.5,116.372402,39.872241,7.1,7.0,7.0,110108,海淀区,110000,北京市,110000,北京市,直辖市,37953406.3859,4440665.70402,0,NaN,NaN


In [54]:
pd.set_option("display.max_columns", 100)
df_2 = pd.read_csv(r'E:\project_basedata\2023data_research\ind_type\new_type_fit_gis_0409.csv',encoding='utf-8',dtype = {'dianping_shops_2021.shop_id_x':'int','dianping_shops_2021.longitude':'float','dianping_shops_2021.latitude':'float'})
df_3 = df_2.loc[:,['dianping_shops_2021.shop_id_x','dianping_shops_2021.longitude','dianping_shops_2021.latitude']]

In [55]:
n = len(df_2)//500000+1
result = np.array_split(df_3,n)

In [81]:
df_i = result[0]
arcpy.env.workspace = r'E:\project_basedata\2023data_research\New File Geodatabase.gdb'
df_i['coord'] = df_i.apply(lambda x: [x['dianping_shops_2021.longitude'], x['dianping_shops_2021.latitude']], axis=1)
df_i

,dianping_shops_2021.shop_id_x,dianping_shops_2021.longitude,dianping_shops_2021.latitude,coord
0,11199358,116.415839,39.909354,"[116.415839, 39.909354]"
1,11228213,116.508680,39.893380,"[116.50868, 39.89338]"
2,11230503,116.508680,39.893380,"[116.50868, 39.89338]"
3,11231113,116.637250,40.064780,"[116.63725, 40.06478]"
4,11231232,116.637250,40.064780,"[116.63725, 40.06478]"
5,11231368,116.637250,40.064780,"[116.63725, 40.06478]"
6,11232655,116.544363,40.099407,"[116.544363, 40.099407]"
7,11232670,116.610061,40.124263,"[116.610061, 40.124263]"
8,11232685,116.660818,40.215644,"[116.660818, 40.215644]"
9,11232694,116.661636,40.108451,"[116.661636, 40.108451]"


In [113]:
df_i_1 = df_i.loc[:,['dianping_shops_2021.shop_id_x','dianping_shops_2021.longitude','dianping_shops_2021.latitude']]
df_i_1

,dianping_shops_2021.shop_id_x,dianping_shops_2021.longitude,dianping_shops_2021.latitude
0,11199358,116.415839,39.909354
1,11228213,116.508680,39.893380
2,11230503,116.508680,39.893380
3,11231113,116.637250,40.064780
4,11231232,116.637250,40.064780
5,11231368,116.637250,40.064780
6,11232655,116.544363,40.099407
7,11232670,116.610061,40.124263
8,11232685,116.660818,40.215644
9,11232694,116.661636,40.108451


In [126]:
lng_lat = df_i_1.values.tolist()
lng_lat

[[11199358.0, 116.41583899999999, 39.909354],
 [11228213.0, 116.50868, 39.89338],
 [11230503.0, 116.50868, 39.89338],
 [11231113.0, 116.63725, 40.06478],
 [11231232.0, 116.63725, 40.06478],
 [11231368.0, 116.63725, 40.06478],
 [11232655.0, 116.54436299999999, 40.099407],
 [11232670.0, 116.61006100000002, 40.124263],
 [11232685.0, 116.66081799999999, 40.215644],
 [11232694.0, 116.66163600000002, 40.108451],
 [11232703.0, 116.55585500000001, 40.077793],
 [11232704.0, 116.70153799999999, 40.098438],
 [11232715.0, 116.700751, 40.095857],
 [11232722.0, 116.65966599999999, 40.131852],
 [11232728.0, 116.60968899999999, 40.126128],
 [11232743.0, 116.66196699999999, 40.135534],
 [11232747.0, 116.55094, 40.0784],
 [11232755.0, 116.65595, 40.14009],
 [11232768.0, 116.652029, 40.174541999999995],
 [11232798.0, 116.82133, 40.160078000000006],
 [11232804.0, 116.715506, 40.040458],
 [11232810.0, 116.65135, 40.13937],
 [11232832.0, 116.716209, 40.040032000000004],
 [11232855.0, 116.652054, 40.138042],

In [2]:
arcpy.env.workspace = r'E:\project_basedata\2023data_research\New File Geodatabase.gdb'
# pd.set_option("display.max_columns", 100)
df_2 = pd.read_csv(r'E:\project_basedata\2023data_research\ind_type\new_type_fit_gis_0409.csv')
point = df_2.loc[:, ['dianping_shops_2021.shop_id_x', 'dianping_shops_2021.longitude', 'dianping_shops_2021.latitude']]

In [4]:
pd.set_option("display.max_columns", 100)
df_2 = pd.read_csv(r'E:\project_basedata\2023data_research\basedata\Export_Output_2_hd.csv')
df_2

,FID,OBJECTID,Join_Count,TARGET_FID,JOIN_FID,dianping_s,dianping_1,dianping_2,dianping_3,dianping_4,dianping_5,dianping_6,dianping_7,dianping_8,dianping_9,dianpin_10,PAC,NAME,省代码_,省_1,市代码_,市_1,类型_1,x,y
0,0,29,1,29,5,11198005,龙泉寺,密云区其他,景点,名胜古迹,0.0,116.091330,40.103298,0.00,0.00,0.00,110108,海淀区,110000,北京市,110000,北京市,直辖市,3.793421e+07,4.453444e+06
1,1,60,1,60,5,11228746,石刻艺术馆,海淀区,景点,展馆展览,0.0,116.330358,39.944725,0.00,0.00,0.00,110108,海淀区,110000,北京市,110000,北京市,直辖市,3.795566e+07,4.437022e+06
2,2,817,1,817,5,11233526,中国移动,清河,生活服务,移动营业厅,0.0,116.362142,40.031223,0.00,0.00,0.00,110108,海淀区,110000,北京市,110000,北京市,直辖市,3.795780e+07,4.446797e+06
3,3,3721,1,3721,5,11237007,Semir,西北旺,购物,服装,3.0,116.224251,40.076557,0.00,0.00,0.00,110108,海淀区,110000,北京市,110000,北京市,直辖市,3.794573e+07,4.451132e+06
4,4,4159,1,4159,5,11237525,东方鸿睿生态农业,通州区,景点,其他景点,0.0,116.287544,39.927243,0.00,0.00,0.00,110108,海淀区,110000,北京市,110000,北京市,直辖市,3.795212e+07,4.434861e+06
5,5,4723,1,4723,5,11238198,国窖1573,梨园,美食,茶饮果汁,0.0,116.368397,39.982037,0.00,0.00,0.00,110108,海淀区,110000,北京市,110000,北京市,直辖市,3.795867e+07,4.441364e+06
6,6,4871,1,4871,5,11238391,天主教,梨园,景点,名胜古迹,3.5,116.255880,40.053440,6.90,6.90,6.90,110108,海淀区,110000,北京市,110000,北京市,直辖市,3.794858e+07,4.448723e+06
7,7,10253,1,10253,5,11244810,安然纳米汗蒸馆,西三旗,休闲娱乐,洗浴/汗蒸,0.0,116.330664,40.056943,0.00,0.00,0.00,110108,海淀区,110000,北京市,110000,北京市,直辖市,3.795494e+07,4.449493e+06
8,8,11562,1,11562,5,11246390,桃花坞旅游风景区,昌平区,景点,自然风光,3.0,116.292563,40.010183,6.90,6.90,6.90,110108,海淀区,110000,北京市,110000,北京市,直辖市,3.795200e+07,4.444102e+06
9,9,11627,1,11627,5,11246462,新龙集贸市场,清河,美食,更多食品保健,0.0,116.329001,40.056223,0.00,0.00,0.00,110108,海淀区,110000,北京市,110000,北京市,直辖市,3.795480e+07,4.449404e+06


In [4]:
# arcpy.env.workspace = r'E:\project_basedata\2023data_research\New File Geodatabase.gdb'
pd.set_option("display.max_columns", 100)
df_2 = pd.read_csv(r'E:\project_basedata\2023data_research\basedata\Export_Output_2_hd.csv',dtype = {'dianping_s':'int'})
df_3 = df_2.head(500)
point = df_3.loc[:, ['dianping_s', 'dianping_6', 'dianping_7']]
# point['coord'] = point.apply(lambda x: [x[1], x[2]], axis=1)
# point_2 = df_2.loc[:, ['x', 'y']]
lng_lat = point.values.tolist()

In [5]:
arcpy.env.workspace = r'E:\project_basedata\2023data_research\basedata'
dsc = arcpy.Describe(r'E:\project_basedata\2023data_research\New File Geodatabase.gdb\Export_xian')
coord_sys = dsc.spatialReference
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference(4525)

In [6]:
out_lay = 'point_bj_hd_3'
arcpy.MakeXYEventLayer_management(table='Export_Output_2_hd.csv', in_x_field='dianping_6', in_y_field='dianping_7', out_layer=out_lay, spatial_reference=coord_sys)

<Result 'point_bj_hd_3'>

In [77]:
# layertofeature
# arcpy.QuickExport_interop??
# arcpy.SaveToLayerFile_management('point_bj_hd_3', 'point_bj_hd_3_layer')
# arcpy.MakeFeatureLayer_management(in_features='point_bj_hd_3_layer',out_layer='point_bj_hd_4')
arcpy.MakeTableView_management(in_table=r'point_bj_hd_3_layer.lyr', out_view='point_bj_hd_3_layer_view')


<Result 'point_bj_hd_3_layer_view'>

In [80]:
arcpy.SelectLayerByAttribute_management(in_layer_or_view='point_bj_hd_3_layer_view')

ExecuteError: ERROR 000339: Input point_bj_hd_3_layer_view does not have OIDs
Failed to execute (SelectLayerByAttribute).


In [57]:
# arcpy.AddField_management(in_table='point_bj_hd_3',field_name='pts_x',field_type='DOUBLE')
# arcpy.CalculateField_management('point_bj_hd_3', 'pts_x','!shape.geodesicArea@METERS!')


ExecuteError: ERROR 000499: table is not editable
Failed to execute (AddField).


In [31]:
arr = arcpy.da.TableToNumPyArray('point_bj_hd_3',('dianping_s','x','y'))
# arr = arcpy.da.FeatureClassToNumPyArray(pt,('POINT_X','POINT_Y'))
df_new_xy = pd.DataFrame(arr)

In [43]:
# arcpy.DefineProjection_management('point_bj_hd_2', coord_sys)
arcpy.AddXY_management('point_bj_hd_3')

ExecuteError: ERROR 000499: table is not editable
Failed to execute (AddXY).


In [32]:
df_new_xy

,dianping_s,x,y
0,11198005,3.793421e+07,4.453444e+06
1,11228746,3.795566e+07,4.437022e+06
2,11233526,3.795780e+07,4.446797e+06
3,11237007,3.794573e+07,4.451132e+06
4,11237525,3.795212e+07,4.434861e+06
5,11238198,3.795867e+07,4.441364e+06
6,11238391,3.794858e+07,4.448723e+06
7,11244810,3.795494e+07,4.449493e+06
8,11246390,3.795200e+07,4.444102e+06
9,11246462,3.795480e+07,4.449404e+06


In [39]:
df_2

,FID,OBJECTID,Join_Count,TARGET_FID,JOIN_FID,dianping_s,dianping_1,dianping_2,dianping_3,dianping_4,dianping_5,dianping_6,dianping_7,dianping_8,dianping_9,dianpin_10,PAC,NAME,省代码_,省_1,市代码_,市_1,类型_1,x,y
0,0,29,1,29,5,11198005,龙泉寺,密云区其他,景点,名胜古迹,0.0,116.091330,40.103298,0.00,0.00,0.00,110108,海淀区,110000,北京市,110000,北京市,直辖市,3.793421e+07,4.453444e+06
1,1,60,1,60,5,11228746,石刻艺术馆,海淀区,景点,展馆展览,0.0,116.330358,39.944725,0.00,0.00,0.00,110108,海淀区,110000,北京市,110000,北京市,直辖市,3.795566e+07,4.437022e+06
2,2,817,1,817,5,11233526,中国移动,清河,生活服务,移动营业厅,0.0,116.362142,40.031223,0.00,0.00,0.00,110108,海淀区,110000,北京市,110000,北京市,直辖市,3.795780e+07,4.446797e+06
3,3,3721,1,3721,5,11237007,Semir,西北旺,购物,服装,3.0,116.224251,40.076557,0.00,0.00,0.00,110108,海淀区,110000,北京市,110000,北京市,直辖市,3.794573e+07,4.451132e+06
4,4,4159,1,4159,5,11237525,东方鸿睿生态农业,通州区,景点,其他景点,0.0,116.287544,39.927243,0.00,0.00,0.00,110108,海淀区,110000,北京市,110000,北京市,直辖市,3.795212e+07,4.434861e+06
5,5,4723,1,4723,5,11238198,国窖1573,梨园,美食,茶饮果汁,0.0,116.368397,39.982037,0.00,0.00,0.00,110108,海淀区,110000,北京市,110000,北京市,直辖市,3.795867e+07,4.441364e+06
6,6,4871,1,4871,5,11238391,天主教,梨园,景点,名胜古迹,3.5,116.255880,40.053440,6.90,6.90,6.90,110108,海淀区,110000,北京市,110000,北京市,直辖市,3.794858e+07,4.448723e+06
7,7,10253,1,10253,5,11244810,安然纳米汗蒸馆,西三旗,休闲娱乐,洗浴/汗蒸,0.0,116.330664,40.056943,0.00,0.00,0.00,110108,海淀区,110000,北京市,110000,北京市,直辖市,3.795494e+07,4.449493e+06
8,8,11562,1,11562,5,11246390,桃花坞旅游风景区,昌平区,景点,自然风光,3.0,116.292563,40.010183,6.90,6.90,6.90,110108,海淀区,110000,北京市,110000,北京市,直辖市,3.795200e+07,4.444102e+06
9,9,11627,1,11627,5,11246462,新龙集贸市场,清河,美食,更多食品保健,0.0,116.329001,40.056223,0.00,0.00,0.00,110108,海淀区,110000,北京市,110000,北京市,直辖市,3.795480e+07,4.449404e+06


In [ ]:
arcpy.SelectLayerByAttribute_management()

In [24]:
arcpy.SaveToLayerFile_management('point_bj_hd_2', r'point_bj_hd_2')

<Result 'E:\\project_basedata\\2023data_research\\basedata\\point_bj_hd_2.lyr'>

In [20]:
arcpy.DefineProjection_management(out_lay, coord_sys)
arcpy.AddXY_management(out_lay)

ExecuteError: ERROR 000289: Failed to alter spatial reference.
Failed to execute (DefineProjection).


In [3]:
arcpy.env.workspace = r'E:\project_basedata\2023data_research\New File Geodatabase.gdb'
pd.set_option("display.max_columns", 100)
df_2 = pd.read_csv(r'E:\project_basedata\2023data_research\basedata\Export_Output_2_hd.csv',dtype = {'dianping_s':'int'})

df_3 = df_2.head(500)
point = df_3.loc[:, ['dianping_s', 'dianping_6', 'dianping_7']]
# point['coord'] = point.apply(lambda x: [x[1], x[2]], axis=1)
# point_2 = df_2.loc[:, ['x', 'y']]
lng_lat = point.values.tolist()
# lng_lat = [x for x in point['coord']]  # 提取坐标

In [4]:
# 根据坐标创建点
pts = arcpy.Point(X=lng_lat[32][1],Y=lng_lat[32][2],ID=lng_lat[32][0])
pt = arcpy.PointGeometry(pts)

# 保存坐标点为实体文件
arcpy.CopyFeatures_management(pt,r'E:\project_basedata\2023data_research\New File Geodatabase.gdb\point_bj_hd')

<Result 'E:\\project_basedata\\2023data_research\\New File Geodatabase.gdb\\point_bj_hd'>

In [5]:
arcpy.env.workspace = r'E:\project_basedata\2023data_research\New File Geodatabase.gdb'
dsc = arcpy.Describe('Export_xian')
coord_sys = dsc.spatialReference
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference(4525)
arcpy.DefineProjection_management('point_bj_hd', coord_sys)
arcpy.AddXY_management('point_bj_hd')

# 将坐标数据输出成dataframe
# arr = arcpy.da.TableToNumPyArray('point_bj_hd',('POINT_X','POINT_Y'))
# arr = arcpy.da.FeatureClassToNumPyArray(pt,('POINT_X','POINT_Y'))
# df_new_xy = pd.DataFrame(arr)

<Result 'E:\\project_basedata\\2023data_research\\New File Geodatabase.gdb\\point_bj_hd'>

In [6]:
arcpy.CopyFeatures_management(pt,r'E:\project_basedata\2023data_research\New File Geodatabase.gdb\point_bj_hd')

ExecuteError: Failed to execute. Parameters are not valid.
ERROR 000725: Output Feature Class: Dataset E:\project_basedata\2023data_research\New File Geodatabase.gdb\point_bj_hd already exists.
Failed to execute (CopyFeatures).


<PointGeometry object at 0x1cee2150[0x1cee2040]>

In [47]:
arcpy.env.workspace = r'E:\project_basedata\2023data_research\New File Geodatabase.gdb'
dsc = arcpy.Describe('Export_xian')
coord_sys = dsc.spatialReference
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference(4525)
pts = [arcpy.Point(ID=int(lng_lat[32][0]),X=lng_lat[32][1],Y=lng_lat[32][2])]
pt = [arcpy.PointGeometry(point) for point in pts]
# array_multi = arcpy.Array(pts)
# point_multi = arcpy.Multipoint(pts)

In [48]:
pt

[<PointGeometry object at 0x3432b530[0x3436f960]>]

In [34]:
point_multi

<geoprocessing describe geometry object object at 0x1C537BE0>


In [30]:
arcpy.AddXY_management(point_multi)

<Result 'in_memory\\f1F618B70_3E14_4D22_88CC_C092B218CDF7'>

In [62]:
arcpy.CopyFeatures_management(pt,r'E:\project_basedata\2023data_research\New File Geodatabase.gdb\point_bj_hd')

<Result 'E:\\project_basedata\\2023data_research\\New File Geodatabase.gdb\\point_bj_hd'>

In [32]:
arr = arcpy.da.FeatureClassToNumPyArray(point_multi,('POINT_X','POINT_Y'))
df_new_xy = pd.DataFrame(arr)

RuntimeError: 'in_table' is not a table or a featureclass

In [27]:
pt

[<PointGeometry object at 0x21ef2ff0[0x21ef2b20]>]

In [28]:
arcpy.CopyFeatures_management(pt,r'E:\project_basedata\2023data_research\New File Geodatabase.gdb\point_bj_hd')

<Result 'E:\\project_basedata\\2023data_research\\New File Geodatabase.gdb\\point_bj_hd'>

In [18]:
arcpy.Type()

AttributeError: 'module' object has no attribute 'Type'

In [17]:
arr = arcpy.da.TableToNumPyArray(pt,('POINT_X','POINT_Y'))
df_new_xy = pd.DataFrame(arr)
df_new_xy

RuntimeError: 'in_table' is not a table or a featureclass

In [ ]:
pt

In [15]:
arcpy.env.workspace = r'E:\project_basedata\2023data_research\New File Geodatabase.gdb'
dsc = arcpy.Describe('Export_xian')
coord_sys = dsc.spatialReference
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference(4525)

In [22]:
arcpy.env.workspace = r'E:\project_basedata\2023data_research\New File Geodatabase.gdb'
dsc = arcpy.Describe('Export_xian')
coord_sys = dsc.spatialReference
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference(4525)
for i in range(len(lng_lat)):
    pts = arcpy.Point(ID=int(lng_lat[i][0]),X=lng_lat[i][1],Y=lng_lat[i][2])
    pt = arcpy.PointGeometry(point)
    arcpy.AddXY_management(pt)

float

In [13]:
type(df_2.loc[234,'dianping_s'])

numpy.int32

In [2]:
arcpy.env.workspace = r'E:\project_basedata\2023data_research\New File Geodatabase.gdb'
pd.set_option("display.max_columns", 100)
df_2 = pd.read_csv(r'E:\project_basedata\2023data_research\basedata\Export_Output_2_hd.csv')
point = df_2.loc[:, ['dianping_s', 'dianping_6', 'dianping_7']]
# point['coord'] = point.apply(lambda x: [x[1], x[2]], axis=1)
# point_2 = df_2.loc[:, ['x', 'y']]
lng_lat = point.values.tolist()
# lng_lat = [x for x in point['coord']]  # 提取坐标

# 根据坐标创建点
pts = [arcpy.Point(X=lng_lat[i][1],Y=lng_lat[i][2],ID=lng_lat[i][0]) for i in range(len(lng_lat))]
pt = [arcpy.PointGeometry(point) for point in pts]

# 保存坐标点为实体文件
arcpy.CopyFeatures_management(pt,r'E:\project_basedata\2023data_research\New File Geodatabase.gdb\point_bj_hd')

<Result 'E:\\project_basedata\\2023data_research\\New File Geodatabase.gdb\\point_bj_hd'>

In [8]:
# 为该坐标点文件定义坐标系
# in_dataset = pt
dsc = arcpy.Describe('Export_xian') # 坐标系来源于2000坐标系的其他文件——县级行政区划图
coord_sys = dsc.SpatialReference
arcpy.DefineProjection_management('point_bj_hd', coord_sys)

# 设置输出的投影坐标系：4525是CGCS_2000_3_Degree_GK_Zone_37的坐标系编码
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference(4525)

# 将原坐标系下经纬度坐标转换为m单位的投影坐标
# in_dataset = 'point_1'
arcpy.AddXY_management('point_bj_hd')

# 将坐标数据输出成dataframe
# arr = arcpy.da.TableToNumPyArray('point_bj_hd',('POINT_X','POINT_Y'))
arr = arcpy.da.FeatureClassToNumPyArray('point_bj_hd',('POINT_X','POINT_Y'))
df_new_xy = pd.DataFrame(arr)

In [9]:
df_new_xy

,POINT_X,POINT_Y
0,3.793421e+07,4.453441e+06
1,3.795568e+07,4.437020e+06
2,3.795780e+07,4.446806e+06
3,3.794573e+07,4.451124e+06
4,3.795211e+07,4.434868e+06
5,3.795868e+07,4.441373e+06
6,3.794858e+07,4.448733e+06
7,3.795494e+07,4.449487e+06
8,3.795199e+07,4.444117e+06
9,3.795481e+07,4.449411e+06


In [5]:
arcpy.env.workspace = r'E:\project_basedata\2023data_research\New File Geodatabase.gdb'
pd.set_option("display.max_columns", 100)
df_2 = pd.read_csv(r'E:\project_basedata\2023data_research\basedata\Export_Output_2_hd.csv')
point = df_2.loc[:, ['FID', 'dianping_6', 'dianping_7']]
# point['coord'] = point.apply(lambda x: [x[1], x[2]], axis=1)
# point_2 = df_2.loc[:, ['x', 'y']]
lng_lat = point.values.tolist()
# lng_lat = [x for x in point['coord']]  # 提取坐标

# 根据坐标创建点
pts = [arcpy.Point(X=lng_lat[i][1],Y=lng_lat[i][2],value_field=lng_lat[i][0]) for i in range(len(lng_lat))]
pt = [arcpy.PointGeometry(point) for point in pts]

# 保存坐标点为实体文件
point_1 = arcpy.CopyFeatures_management(pt,r'E:\project_basedata\2023data_research\New File Geodatabase.gdb\point_bj_hd')

# 为该坐标点文件定义坐标系
in_dataset = 'point_bj_hd'
dsc = arcpy.Describe('Export_xian') # 坐标系来源于2000坐标系的其他文件——县级行政区划图
coord_sys = dsc.spatialReference
arcpy.DefineProjection_management(in_dataset, coord_sys)

# 设置输出的投影坐标系：4525是CGCS_2000_3_Degree_GK_Zone_37的坐标系编码
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference(4525)

# 将原坐标系下经纬度坐标转换为m单位的投影坐标
# in_dataset = 'point_1'
arcpy.AddXY_management(in_dataset)

# 将坐标数据输出成dataframe
arr = arcpy.da.TableToNumPyArray('point_bj_hd',('POINT_X','POINT_Y'))
df_new_xy = pd.DataFrame(arr)
# in_table = 'point_1'
# out_xls = 'E:\project_basedata\2023data_research\basedata\point_bj_hd.xls'
# arcpy.TableToExcel_conversion(in_table, out_xls)

<Result 'E:\\project_basedata\\2023data_research\\New File Geodatabase.gdb\\point_bj_hd'>

In [79]:
array = np.array([(1, (471316.3835861763, 5000448.782036674)),
                     (2, (470402.49348005146, 5000049.216449278))],
                    np.dtype([('idfield',np.int32),('XY', '<f8', 2)]))

In [80]:
array

array([(1, [471316.3835861763, 5000448.782036674]),
       (2, [470402.49348005146, 5000049.216449278])], 
      dtype=[('idfield', '<i4'), ('XY', '<f8', (2,))])

In [32]:
df_2 = pd.read_csv(r'E:\project_basedata\2023data_research\basedata\Export_Output_2_hd.csv',encoding='utf-8')
df_4 = df_2.loc[:,['FID','dianping_6','dianping_7']]
df_3 = df_2.loc[:,['FID','OBJECTID','dianping_3','dianping_4','dianping_6','dianping_7']]
df_3['coord'] = df_3.apply(lambda x: [x[4], x[5]], axis=1)
df_4['coord'] = df_4.apply(lambda x: [x[1], x[2]], axis=1)

In [35]:
array2 = np.array(df_4)
array2

array([[0L, 116.09133000000001, 40.103298,
        [116.09133000000001, 40.103298000000002]],
       [1L, 116.330358, 39.944725, [116.330358, 39.944724999999998]],
       [2L, 116.362142, 40.031223, [116.36214200000001, 40.031222999999997]],
       ..., 
       [250445L, 116.140358, 39.726384,
        [116.14035800000001, 39.726384000000003]],
       [250446L, 116.140358, 39.726384,
        [116.14035800000001, 39.726384000000003]],
       [250447L, 116.140358, 39.726384,
        [116.14035800000001, 39.726384000000003]]], dtype=object)

In [31]:
array1 = np.array(df_3,np.dtype([('idfield',np.int32),('idfield1',np.int32),('type1',np.str),('type2',np.str),('x',np.float32),('y',np.float32),('xy','<f8',2)]))
array1

TypeError: expected a readable buffer object

In [38]:
arcpy.env.workspace = r'E:\project_basedata\2023data_research\New File Geodatabase.gdb'
outFC = 'E:\project_basedata\2023data_research\New File Geodatabase.gdb\point_bj_hd_1'
dsc = arcpy.Describe('Export_xian') # 坐标系来源于2000坐标系的其他文件——县级行政区划图
coord_sys = dsc.spatialReference
arcpy.da.NumPyArrayToFeatureClass(array2, outFC, [-1], coord_sys)

TypeError: narray.fields require

In [19]:
array1[0][-2]

116.09133000000001

In [16]:
array1 = np.array(df_3)
array1

array([[0L, 29L, u'\u666f\u70b9', u'\u540d\u80dc\u53e4\u8ff9',
        116.09133000000001, 40.103298],
       [1L, 60L, u'\u666f\u70b9', u'\u5c55\u9986\u5c55\u89c8', 116.330358,
        39.944725],
       [2L, 817L, u'\u751f\u6d3b\u670d\u52a1',
        u'\u79fb\u52a8\u8425\u4e1a\u5385', 116.362142, 40.031223],
       ..., 
       [250445L, 1705245L, u'\u7f8e\u98df', u'\u7c89\u9762\u9986',
        116.140358, 39.726384],
       [250446L, 1705247L, u'\u7f8e\u98df', u'\u7c89\u9762\u9986',
        116.140358, 39.726384],
       [250447L, 1705251L, u'\u7f8e\u98df', u'\u7c89\u9762\u9986',
        116.140358, 39.726384]], dtype=object)

In [55]:
array

array([(1, [471316.3835861763, 5000448.782036674]),
       (2, [470402.49348005146, 5000049.216449278])], 
      dtype=[('idfield', '<i4'), ('XY', '<f8', (2,))])

In [ ]:
arcpy.da.NumPyArrayToFeatureClass(array, outFC, ['XY'], SR)

In [28]:
filteredFields = arcpy.da.ListFieldConflictFilters('point_bj_hd')

In [47]:
arr = arcpy.da.TableToNumPyArray('point_bj_hd',('POINT_X','POINT_Y'))

In [50]:
df1 = pd.DataFrame(arr)
df1.iloc[32,0]

37957408.105951183

In [31]:
input1 = arcpy.GetParameterAsText(0)
field1 = arcpy.GetParameterAsText(1)
field2 = arcpy.GetParameterAsText(2)
arr = arcpy.da.TableToNumPyArray(input1, (field1, field2))
print(numpy.corrcoef((arr[field1], arr[field2])))

RuntimeError: cannot open '-f'

In [27]:
in_dataset

'point_bj_hd'

In [26]:
arcpy.Export_arc(feature_type='PLOT',compression_type='NONE',in_dataset='point_bj_hd', interchange_file='pointbjhd.csv')

ExecuteError: Failed to execute. Parameters are not valid.
ERROR 000815: The value is not a file
Failed to execute (Export).


In [22]:
arcpy.Export_arc??

In [19]:
arcpy.QuickExport_interop(Input='point_bj_hd',Output='pointbjhd.csv')

RuntimeError: Object: Error in executing tool

In [20]:
arcpy.TableToTable_conversion(in_rows='point_bj_hd', out_path='E:\project_basedata\2023data_research\basedata',out_name='pointbjhd.csv')

ExecuteError: Failed to execute. Parameters are not valid.
ERROR 000735: Output Location: Value is required
Failed to execute (TableToTable).


In [50]:
arcpy.env.workspace = r'E:\project_basedata\2023data_research\New File Geodatabase.gdb'
# pd.set_option("display.max_columns", 100)
df_2 = pd.read_csv(r'E:\project_basedata\2023data_research\ind_type\new_type_fit_gis_0409.csv')
point = df_2.loc[:, ['dianping_shops_2021.shop_id_x', 'dianping_shops_2021.longitude', 'dianping_shops_2021.latitude']]
point['coord'] = point.apply(lambda x: [x[1], x[2]], axis=1)
# point_2 = df_2.loc[:, ['x', 'y']]
lng_lat = [x for x in point['coord']]  # 提取坐标

# 根据坐标创建点
pts = [arcpy.Point(*cood) for cood in lng_lat]
pt = [arcpy.PointGeometry(point) for point in pts]

# 保存坐标点为实体文件
point_1 = arcpy.CopyFeatures_management(pt,r'E:\project_basedata\2023data_research\New File Geodatabase.gdb\point_bj')

# 为该坐标点文件定义坐标系
in_dataset = 'point_bj'
dsc = arcpy.Describe('Export_xian') # 坐标系来源于2000坐标系的其他文件——县级行政区划图
coord_sys = dsc.spatialReference
arcpy.DefineProjection_management(in_dataset, coord_sys)

# 设置输出的投影坐标系：4525是CGCS_2000_3_Degree_GK_Zone_37的坐标系编码
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference(4525)

# 将原坐标系下经纬度坐标转换为m单位的投影坐标
# in_dataset = 'point_1'
arcpy.AddXY_management(in_dataset)

# 将数据输出成表格
# in_table = 'point_1'
# out_xls = 'E:\project_basedata\2023data_research\basedata\point_bj_hd.xls'
# arcpy.TableToExcel_conversion(in_table, out_xls)

MemoryError: 

In [47]:
# 将数据输出成表格
in_table = 'point_1'
# out_xls = 'E:\project_basedata\2023data_research\basedata\point_bj_hd.csv'
outLocation = 'E:/project_basedata/2023data_research/basedata'
outTable = 'point_bj_hd.csv'

# arcpy.TableToExcel_conversion(in_table, out_xls)
# arcpy.TableToDBASE_conversion(in_table, outLocation)
arcpy.TableToTable_conversion(in_table, outLocation, outTable, expression)

<Result 'E:\\project_basedata\\2023data_research\\basedata'>

In [29]:
arcpy.Describe('Export_example')

<geoprocessing describe data object at 0x314eaea8>

In [35]:
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference(4525)
# 

In [37]:
in_dataset = 'point_1'
# in_features = 'point_2'
# arcpy.Copy_management(in_dataset, in_features)
arcpy.AddXY_management(in_dataset)

<Result 'E:\\project_basedata\\2023data_research\\New File Geodatabase.gdb\\point_1'>

In [11]:
point_1 = arcpy.CopyFeatures_management(pt,r'E:\project_basedata\2023data_research\New File Geodatabase.gdb\point_1')

In [16]:
in_dataset = 'point_1'
dsc = arcpy.Describe('Export_xian')

In [18]:
coord_sys = dsc.spatialReference
coord_sys

<geoprocessing spatial reference object at 0x31633d58>

In [20]:
arcpy.DefineProjection_management(in_dataset, coord_sys)

<Result 'E:\\project_basedata\\2023data_research\\New File Geodatabase.gdb\\point_1'>

In [15]:
arcpy.DefineProjection_management(point_1, 'E:\project_basedata\2023data_research\CGCS_2000.prj')

ExecuteError: Failed to execute. Parameters are not valid.
ERROR 000735: Coordinate System: Value is required
Failed to execute (DefineProjection).


In [12]:
arcpy.Describe(point_1).spatialReference

<geoprocessing spatial reference object at 0x3120f998>

In [45]:
sr_p = arcpy.SpatialReference('E:/project_basedata/2023data_research/CGCS_2000_3_Degree_GK_Zone_37.prj')

RuntimeError: ERROR 999999: Error executing function.

In [44]:
sr = arcpy.SpatialReference('E:/project_basedata/2023data_research/CGCS_2000.prj')

RuntimeError: ERROR 999999: Error executing function.

In [7]:
arcpy.SpatialReference??